## Segmenting and clustering neighborhoods - Toronto
## Task 3 - Exploring and Clustering
### See section "Exploring and Clustering"

The first few cells are a repetition from task 1 and task 2.

Importing the libraries including BeautifulSoup for web scraping tasks:

In [117]:
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup

# use module dotenv to manage API keys and secrets
import os
%load_ext dotenv
%dotenv

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.geocoders import ArcGIS # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
Libraries imported.


## Data gathering
Collect data from wikipedia with beautiful soup:

In [118]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(wiki_url).text
soup = BeautifulSoup(website_url,'html.parser')
table = soup.find('table', attrs={'class':'wikitable sortable'})
rows = table.find_all('tr')[1:]
list_codes = []
for row in rows:
    cols = row.find_all('td')
    list_codes.append({'PostalCode':cols[0].text, 'Borough':cols[1].text, 'Neighborhood':cols[2].text})

df_nbh = pd.DataFrame(list_codes)

# there are line breaks in the neighborhood column, get rid of them:
df_nbh.Neighborhood = df_nbh.Neighborhood.str.replace('\\n','')

# drop not assigned boroughs
df_nbh = df_nbh.replace({'Not assigned': np.nan})
df_nbh.dropna(subset = ['Borough'],inplace=True)

#reset index:
df_nbh.reset_index(drop=True, inplace = True)
df_nbh.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### group the data by the first 2 columns and combine the neighborhood values into a comma seperated string:

In [119]:
df_nbh = df_nbh.groupby(['PostalCode', 'Borough'], as_index=False).apply(lambda x: ', '.join(map(str,list(x.Neighborhood)))).reset_index()
df_nbh.rename(columns={0:'Neighborhood'}, inplace= True)
df_nbh.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


replace missing neighborhood data with borough:

In [120]:
# replace missing Neighborhood data with Borough:
df_nbh.loc[df_nbh.Neighborhood == 'nan', 'Neighborhood' ] = df_nbh.Borough
df_nbh.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [121]:
df_nbh.shape

(103, 3)

### Geospatial data
ArcGIS turned out to be the most reliable geocoding service in the geopy library: 

In [122]:
geolocator = ArcGIS(user_agent="toronto_neighborhood")
# initialize location column:
df_nbh['location'] = np.nan
df_nbh['location'] = df_nbh[df_nbh.location.isna()].apply(lambda x : geolocator.geocode( "Toronto, Ontario,  " +  str(x['PostalCode'])), axis=1)
df_nbh['location'].head()

0    (M1B, (43.811525000000074, -79.19551746399998))
1     (M1C, (43.78566500000005, -79.15872457299997))
2     (M1E, (43.76581500000003, -79.17519294699997))
3    (M1G, (43.768369121000035, -79.21758999999997))
4     (M1H, (43.76968799900004, -79.23943999999995))
Name: location, dtype: object

In [123]:
# are there any nans left?
df_nbh['location'].isna().sum()

0

now we need to create the columns for longitude and latitude:


In [124]:
# defining functions to use in apply
def get_longitude(location):
    try:
        return location.longitude
    except Exception:
        return np.nan
def get_latitude(location):
    try:
        return location.latitude
    except Exception:
        return np.nan
    
# use apply to fill the new columns:
df_nbh['Longitude']= df_nbh.location.apply(get_longitude)
df_nbh['Latitude']= df_nbh.location.apply(get_latitude)
df_nbh.head()

,PostalCode,Borough,Neighborhood,location,Longitude,Latitude
0,M1B,Scarborough,"Rouge, Malvern","(M1B, (43.811525000000074, -79.19551746399998))",-79.195517,43.811525
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union","(M1C, (43.78566500000005, -79.15872457299997))",-79.158725,43.785665
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","(M1E, (43.76581500000003, -79.17519294699997))",-79.175193,43.765815
3,M1G,Scarborough,Woburn,"(M1G, (43.768369121000035, -79.21758999999997))",-79.217590,43.768369
4,M1H,Scarborough,Cedarbrae,"(M1H, (43.76968799900004, -79.23943999999995))",-79.239440,43.769688


In [125]:
df_nbh.drop(columns=['location'], inplace=True)
df_nbh.reset_index(drop=True, inplace=True)

In [126]:
df_nbh.head()

,PostalCode,Borough,Neighborhood,Longitude,Latitude
0,M1B,Scarborough,"Rouge, Malvern",-79.195517,43.811525
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",-79.158725,43.785665
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",-79.175193,43.765815
3,M1G,Scarborough,Woburn,-79.217590,43.768369
4,M1H,Scarborough,Cedarbrae,-79.239440,43.769688


In [127]:
# create map of Toronto using latitude and longitude values
df_toronto = df_nbh.copy()
loc_toronto = geolocator.geocode('Toronto')

map_toronto = folium.Map(location=[loc_toronto.latitude, loc_toronto.longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Exploring and Clustering

#### Setting up Foursquare API keys:

In [128]:
client_id = os.environ.get('CLIENT_ID')
client_secret = os.environ.get('CLIENT_SECRET')
version = '20180605'

#### Define function for getting nearby venues for given location data

In [129]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Get Boroughs that contain word 'Toronto' 

In [130]:
df_boroughs = df_toronto[df_toronto.Borough.str.match('.*Toronto.*')].reset_index()
df_boroughs

,index,PostalCode,Borough,Neighborhood,Longitude,Latitude
0,37,M4E,East Toronto,The Beaches,-79.295425,43.676531
1,41,M4K,East Toronto,"The Danforth West, Riverdale",-79.355105,43.683178
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",-79.314667,43.667965
3,43,M4M,East Toronto,Studio District,-79.334855,43.660629
4,44,M4N,Central Toronto,Lawrence Park,-79.387133,43.728420
5,45,M4P,Central Toronto,Davisville North,-79.388514,43.712755
6,46,M4R,Central Toronto,North Toronto West,-79.406960,43.714523
7,47,M4S,Central Toronto,Davisville,-79.385964,43.703395
8,48,M4T,Central Toronto,"Moore Park, Summerhill East",-79.382946,43.690685
9,49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",-79.402265,43.686074


##### Get the 100 most nearby venues for all neighborhoods in df_boroughs 

In [131]:
df_toronto_venues = getNearbyVenues(df_boroughs.Neighborhood, df_boroughs.Latitude, df_boroughs.Longitude)

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [132]:
df_toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676531,-79.295425,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676531,-79.295425,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676531,-79.295425,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676531,-79.295425,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.683178,-79.355105,Dollarama,43.686197,-79.355989,Discount Store
5,"The Danforth West, Riverdale",43.683178,-79.355105,Charles Sauriol Parkette,43.685270,-79.356588,Park
6,"The Danforth West, Riverdale",43.683178,-79.355105,TTC Bus #8 Broadview,43.687101,-79.355078,Bus Line
7,"The Danforth West, Riverdale",43.683178,-79.355105,Sobeys,43.684723,-79.356461,Grocery Store
8,"The Beaches West, India Bazaar",43.667965,-79.314667,System Fitness,43.667171,-79.312733,Gym
9,"The Beaches West, India Bazaar",43.667965,-79.314667,Brett's Ice Cream,43.667222,-79.312831,Ice Cream Shop


#### Analyze each neighborhood:

In [133]:
# one hot encoding
df_toronto_onehot = pd.get_dummies(df_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframed
df_toronto_onehot['Neighborhood'] = df_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [df_toronto_onehot.columns[-1]] + list(df_toronto_onehot.columns[:-1])
#df_toronto_onehot = df_toronto_onehot[fixed_columns]

df_nb = df_toronto_onehot['Neighborhood']
df_toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
df_toronto_onehot.insert(0, 'Neighborhood', df_nb)

df_toronto_onehot.sample(10)

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
65,Studio District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1049,"Chinatown, Grange Park, Kensington Market",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [134]:
# examine the size:
df_toronto_onehot.shape

(1772, 221)

#### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [135]:
df_toronto_grouped = df_toronto_onehot.groupby('Neighborhood').mean().reset_index()
df_toronto_grouped.sample(5)

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
23,"Little Portugal, Trinity",0.0,0.019231,0.0,0.019231,0.0,0.057692,0.0,0.00,0.0,0.0,0.019231,0.00,0.076923,0.00,0.00,0.019231,0.0,0.019231,0.0,0.00,0.019231,0.0,0.00,0.019231,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.057692,0.057692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.019231,0.019231,0.0,0.019231,0.00,0.0,0.0,0.0,0.019231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.019231,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.019231,0.00,0.0,0.019231,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.019231,0.0,0.0,0.0,0.019231,0.019231,0.00,0.0,0.019231,0.0,0.019231,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.019231,0.00,0.0,0.019231,0.0,0.0,0.00,0.0,0.00,0.0,0.019231,0.019231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019231,0.0,0.0,0.0,0.0,0.0,0.0,0.

#### Let's print each neighborhood along with the top 3 most common venues

In [136]:
num_top_venues = 3

for hood in df_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_toronto_grouped[df_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2   Steakhouse  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1  Cocktail Bar  0.05
2        Bakery  0.03


----Brockton, Exhibition Place, Parkdale Village----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.07
2   Restaurant  0.06


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.09
1   Steakhouse  0.04
2        Hotel  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.07
2                Café  0.06


----Cabbagetown, St. James Town----
         venue  freq
0         Café  0.08
1  Coffee Shop  0.08
2       Bakery  0.05


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.11
1  Clothing Store  0.07
2           Plaza  0.03


----Chinatow

#### Let's put that into a *pandas* dataframe

define function to sort venues in descending order:

In [137]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [138]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_toronto_grouped['Neighborhood']

for ind in np.arange(df_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Bar,Bakery,Breakfast Spot,Restaurant,Japanese Restaurant,Gym
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Restaurant,Farmers Market,Bakery,Beer Bar,Breakfast Spot,Cheese Shop,Café
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Furniture / Home Store,Restaurant,Bakery,Bar,Hotel,Italian Restaurant,Vegetarian / Vegan Restaurant,Sandwich Place
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Steakhouse,Hotel,Seafood Restaurant,Café,Sushi Restaurant,Gym,Pub,Bar,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Café,Bar,Restaurant,Pub,Bakery,Gym / Fitness Center,Electronics Store,Park


### Clustering the data
Run k-means to cluster the neighborhoods into 5 clusters:

In [139]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = df_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [140]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 2, 0,
       3, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [141]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_boroughs


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37,M4E,East Toronto,The Beaches,-79.295425,43.676531,0.0,Health Food Store,Pub,Trail,Yoga Studio,Dumpling Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,41,M4K,East Toronto,"The Danforth West, Riverdale",-79.355105,43.683178,1.0,Bus Line,Grocery Store,Park,Discount Store,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",-79.314667,43.667965,0.0,Park,Sandwich Place,Pizza Place,Italian Restaurant,Pub,Movie Theater,Fast Food Restaurant,Fish & Chips Shop,Pet Store,Burger Joint
3,43,M4M,East Toronto,Studio District,-79.334855,43.660629,0.0,Diner,Brewery,Italian Restaurant,Café,Sandwich Place,Bar,Coffee Shop,Pizza Place,Gastropub,Sushi Restaurant
4,44,M4N,Central Toronto,Lawrence Park,-79.387133,43.728420,3.0,Bus Line,Swim School,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [142]:
# create map
map_clusters = folium.Map(location=[loc_toronto.latitude, loc_toronto.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
toronto_merged.dropna(subset=['Cluster Labels'], inplace=True)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [143]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,43.676531,0.0,Health Food Store,Pub,Trail,Yoga Studio,Dumpling Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,M4L,43.667965,0.0,Park,Sandwich Place,Pizza Place,Italian Restaurant,Pub,Movie Theater,Fast Food Restaurant,Fish & Chips Shop,Pet Store,Burger Joint
3,M4M,43.660629,0.0,Diner,Brewery,Italian Restaurant,Café,Sandwich Place,Bar,Coffee Shop,Pizza Place,Gastropub,Sushi Restaurant
5,M4P,43.712755,0.0,Food & Drink Shop,Park,Hotel,Department Store,Gym,Breakfast Spot,Japanese Restaurant,Event Space,Falafel Restaurant,Farm
7,M4S,43.703395,0.0,Dessert Shop,Pizza Place,Sandwich Place,Coffee Shop,Italian Restaurant,Café,Gas Station,Diner,Restaurant,Park
8,M4T,43.690685,0.0,Gym,Convenience Store,Trail,Yoga Studio,Dumpling Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
9,M4V,43.686074,0.0,Light Rail Station,Coffee Shop,Supermarket,Liquor Store,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
11,M4X,43.668160,0.0,Café,Coffee Shop,Pizza Place,Restaurant,Italian Restaurant,Bakery,Park,Diner,Market,Breakfast Spot
12,M4Y,43.666585,0.0,Coffee Shop,Restaurant,Gay Bar,Japanese Restaurant,Sushi Restaurant,Café,Hotel,Fast Food Restaurant,Dance Studio,Gastropub
13,M5A,43.650295,0.0,Coffee Shop,Bakery,Boat or Ferry,Theater,Café,Gym / Fitness Center,Shoe Store,Gastropub,Brewery,Spa


### Cluster 2

In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4K,43.683178,1.0,Bus Line,Grocery Store,Park,Discount Store,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
6,M4R,43.714523,1.0,Playground,Gym Pool,Park,Garden,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
10,M4W,43.682205,1.0,Playground,Candy Store,Grocery Store,Park,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


### Cluster 3

In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5J,43.63021,2.0,Pier,Harbor / Marina,Park,Yoga Studio,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


### Cluster 4

In [146]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,43.72842,3.0,Bus Line,Swim School,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


### Cluster 5

In [147]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,M5P,43.694785,4.0,Park,Yoga Studio,Dumpling Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
